# Классификация (1-ая часть)

### Постановка задачи: 
#### Обучающая выборка разбита на два класса: положительный и отрицательный. Данные для обучения описываются положительным и отрицательным контекстами. Необходимо классифицировать новые объекты либо в положительный, либо в отрицательный класс, либо же отказаться от классификации. 

In [3]:
import numpy as np
import pandas as pd


def f(s, graph):
    global ans_graph
    global repeat
    if s in repeat:
        ans_graph = 0
        return
    for g in graph:
        if g[0] == s:
            f(g[1], graph)
    return

#### Введем данные для положительного класса

In [ ]:
n_plus, m_plus = map(int, input().split())
mas_plus = np.empty((n_plus, m_plus), dtype=int)
for i in range(n_plus):
    mas_plus[i] = np.array(list(map(int, input().split())))

#### И для отрицательного класса

In [ ]:
n_minus, m_minus = map(int, input().split())
mas_minus = np.empty((n_minus, m_minus), dtype=int)
for i in range(n_minus):
    mas_minus[i] = np.array(list(map(int, input().split())))

#### Теперь построим решетку формальных понятий для положительного класса. Для этого пройдемся по всем возможным объединениям объектов обучающей выборки и построим для них формальные содержания. Из двух формальных объемов с двумя одинаковыми формальными содержаниями выбирается тот, у кого объем больше. После этого строится граф. Для каждой формального понятия отыскиваются такие другие понятия, что объем изначального является подмножеством объемов отыскиваемых. Далее проверяем, что между двумя вершинами уже нет пути, и соединяем их 

In [ ]:
lattice_plus = list([(np.array([], dtype=int), np.array([1 for i in range(m_plus)]))])
graph_plus = []
hypotheses_plus = np.empty((0, m_plus))
ans_graph = 0
current = 1


for i in range(1, 1 << n_plus):
    vector = np.ones(m_plus, dtype=int)
    arr = np.array([], dtype=int)
    mis = np.array([], dtype=int)
    y = 1
    for j in range(n_plus):
        if i % (2*y) // y:
            vector = np.bitwise_and(vector, mas_plus[j])
            arr = np.append(arr, j)
        y *= 2
    for j in range(n_plus):
        if (np.bitwise_and(vector, mas_plus[j]) == vector).all():
            mis = np.append(mis, j)
    y = 0
    if np.array_equal(mis, arr):
        repeat = set()
        for j in range(len(lattice_plus) - 1, -1, -1):
            if set(lattice_plus[j][0]) <= set(arr):
                ans_graph = 1
                f(j, graph_plus)
                yy = ans_graph
                if yy:
                    graph_plus.append((j, current))
                    repeat.add(j)
                    y = 1
        if y:
            lattice_plus.append((arr, vector))
            current += 1

#### И также построим решетку для отрицательного класса

In [ ]:
lattice_minus = list([(np.array([], dtype=int), np.array([1 for i in range(m_minus)]))])
graph_minus = []
hypotheses_minus = np.empty((0, m_minus))
ans_graph = 0
current = 1


for i in range(1, 1 << n_minus):
    vector = np.ones(m_minus, dtype=int)
    arr = np.array([], dtype=int)
    mis = np.array([], dtype=int)
    y = 1
    for j in range(n_minus):
        if i % (2*y) // y:
            vector = np.bitwise_and(vector, mas_minus[j])
            arr = np.append(arr, j)
        y *= 2
    for j in range(n_minus):
        if (np.bitwise_and(vector, mas_minus[j]) == vector).all():
            mis = np.append(mis, j)
    y = 0
    if np.array_equal(mis, arr):
        repeat = set()
        for j in range(len(lattice_minus) - 1, -1, -1):
            if set(lattice_minus[j][0]) <= set(arr):
                ans_graph = 1
                f(j, graph_minus)
                yy = ans_graph
                if yy:
                    graph_minus.append((j, current))
                    repeat.add(j)
                    y = 1
        if y:
            lattice_minus.append((arr, vector))
            current += 1

#### Теперь сформируем гипотезы. Для этого сначала соберем все содержания формальных понятий и исключим оттуда все фальсифицированные гипотезы, то есть те, которые являются подмножествами какого-то содержания понятия другого класса. Затем оставим лишь те гипотезы, которые не включают в себя какие-то другие гипотезы, то есть выберем минимальные гипотезы

In [ ]:
for_delete = set()
for i in lattice_plus:
    hypotheses_plus = np.vstack((hypotheses_plus, i[1]))
for i in range(len(hypotheses_plus)):
    for j in lattice_minus[1:]:
        if (hypotheses_plus[i] <= j[1]).all():
            for_delete.add(i)
for_delete = list(for_delete)
for i in for_delete[::-1]:
    hypotheses_plus = np.delete(hypotheses_plus, i, 0)
for_delete = set()
for i in range(len(hypotheses_plus)):
    for j in range(len(hypotheses_plus)):
        if i != j and (hypotheses_plus[i] >= hypotheses_plus[j]).all():
            for_delete.add(i)
for_delete = list(for_delete)
for i in for_delete[::-1]:
    hypotheses_plus = np.delete(hypotheses_plus, i, 0)
    

for_delete = set()
for i in lattice_minus:
    hypotheses_minus = np.vstack((hypotheses_minus, i[1]))
for i in range(len(hypotheses_minus)):
    for j in lattice_plus[1:]:
        if (hypotheses_minus[i] <= j[1]).all():
            for_delete.add(i)
for_delete = list(for_delete)
for i in for_delete[::-1]:
    hypotheses_minus = np.delete(hypotheses_minus, i, 0)
for_delete = set()
for i in range(len(hypotheses_minus)):
    for j in range(len(hypotheses_minus)):
        if i != j and (hypotheses_minus[i] >= hypotheses_minus[j]).all():
            for_delete.add(i)
for_delete = list(for_delete)
for i in for_delete[::-1]:
    hypotheses_minus = np.delete(hypotheses_minus, i, 0)

#### Теперь мы можем классифицировать новые объекты

In [ ]:
num = int(input())
for i in range(num):
    obj = np.array(list(map(int, input().split())))
    q = 0
    for j in hypotheses_plus:
        if (j <= obj).all():
            q = 1
    for j in hypotheses_minus:
        if (j <= obj).all() and q != 1:
            q = -1
        elif (j <= obj).all():
            q = 2
    if q == 1:
        print("PLUS")
    elif q == -1:
        print("MINUS")
    else:
        print("DENIED")

#### Пример 1
#### Диагностика COVID-19
#### Даны два списка пациентов с описанием симптомов каждого. У пациентов первого списка подтверждено заболевание короновирусной инфекцией, у пациентов из второго списка подозрения на короновирус не подтвердились. Цель: выявить те симптомы, которые сильнее других указывают на заболевание короновирусом и поставить диагноз новым пациентам
#### Первая таблица - список пациентов с подтвердившимся диагнозом, вторая таблица - список пациентов, чей диагноз не подтвердился

In [18]:
example1 = pd.DataFrame({'повышение температуры': ['+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '-', '+'], 'боль в горле': ['+', '+', '-', '+', '-', '+', '+', '+', '+', '-', '+', '+'], 'головная боль': ['-', '-', '+', '+', '+', '-', '+', '+', '-', '-', '-', '+'], 'кашель': ['+', '-', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+'], 'покраснение глаз': ['+', '-', '-', '-', '-', '+', '+', '+', '-', '-', '-', '-'], 'сыпь на коже': ['-', '+', '-', '-', '-', '-', '+', '+', '-', '-', '-', '-'], 'утомляемость': ['+', '+', '+', '+', '-', '-', '+', '+', '+', '+', '+', '-']})
example1 = example1.style.set_properties(**{'font-size': '16pt'})
example1

,повышение температуры,боль в горле,головная боль,кашель,покраснение глаз,сыпь на коже,утомляемость
0,+,+,-,+,+,-,+
1,+,+,-,-,-,+,+
2,+,-,+,+,-,-,+
3,+,+,+,+,-,-,+
4,+,-,+,+,-,-,-
5,+,+,-,+,+,-,-
6,+,+,+,+,+,+,+
7,+,+,+,+,+,+,+
8,+,+,-,+,-,-,+
9,+,-,-,+,-,-,+


In [19]:
example2 = pd.DataFrame({'повышение температуры': ['+', '-', '-', '-', '+', '+', '+', '-', '-'], 'боль в горле': ['-', '-', '-', '-', '+', '-', '-', '-', '+'], 'головная боль': ['-', '+', '-', '-', '-', '-', '-', '+', '-'], 'кашель': ['+', '+', '-', '-', '+', '-', '-', '-', '-'], 'покраснение глаз': ['-', '-', '-', '+', '-', '+', '-', '-', '-'], 'сыпь на коже': ['-', '-', '-', '+', '-', '-', '-', '-', '-'], 'утомляемость': ['-', '-', '+', '-', '-', '-', '-', '-', '-']})
example2 = example2.style.set_properties(**{'font-size': '16pt'})
example2

,повышение температуры,боль в горле,головная боль,кашель,покраснение глаз,сыпь на коже,утомляемость
0,+,-,-,+,-,-,-
1,-,-,+,+,-,-,-
2,-,-,-,-,-,-,+
3,-,-,-,-,+,+,-
4,+,+,-,+,-,-,-
5,+,-,-,-,+,-,-
6,+,-,-,-,-,-,-
7,-,-,+,-,-,-,-
8,-,+,-,-,-,-,-


#### Построив две решетки и сформировав гипотезы, получаем 5 положительных гипотез:
##### повышение температуры И боль в горле И кашель И покраснение глаз
##### повышение температуры И утомляемость
##### боль в горле И утомляемость
##### кашель И утомляемость
##### повышене температуры И головная боль И кашель И утомляемость
#### Теперь попробуем поставить диагноз новым пациентам исходя из данных гипотез:

In [20]:
example3 = pd.DataFrame({'повышение температуры': ['+', '+', '+', '-', '+', '+', '-', '-'], 'боль в горле': ['+', '+', '+', '-', '-', '+', '-', '-'], 'головная боль': ['+', '-', '-', '+', '-', '+', '-', '-'], 'кашель': ['+', '-', '-', '+', '-', '+', '-', '+'], 'покраснение глаз': ['-', '+', '-', '-', '-', '+', '-', '+'], 'сыпь на коже': ['-', '+', '-', '-', '-', '-', '+', '-'], 'утомляемость': ['-', '-', '+', '+', '-', '-', '+', '+'], 'ДИАГНОЗ': ['БОЛЕН', 'доп.обследование', 'БОЛЕН', 'БОЛЕН', 'доп.обследование', 'БОЛЕН', 'доп.обследование', 'БОЛЕН']})
example3 = example3.style.set_properties(**{'font-size': '16pt'})
example3

,повышение температуры,боль в горле,головная боль,кашель,покраснение глаз,сыпь на коже,утомляемость,ДИАГНОЗ
0,+,+,+,+,-,-,-,БОЛЕН
1,+,+,-,-,+,+,-,доп.обследование
2,+,+,-,-,-,-,+,БОЛЕН
3,-,-,+,+,-,-,+,БОЛЕН
4,+,-,-,-,-,-,-,доп.обследование
5,+,+,+,+,+,-,-,БОЛЕН
6,-,-,-,-,-,+,+,доп.обследование
7,-,-,-,+,+,-,+,БОЛЕН


#### Пример 2
#### Выдача займа
#### Даны два списка клиентов банка и их характеристики. Клиентам из первого списка займ был одобрен, клиентам из второго списка - отклонен. Цель: выявить те признаки, которые сильнее других влияют на одобрение займа и решить, выдавать займ новым клиентам или нет
#### Первая таблица - список клентов, кому был одобрен займ, вторая таблица - те, кому займ был отклонен

In [4]:
example4 = pd.DataFrame({'высшее образование': ['-', '+', '-', '-', '+', '-', '-', '+', '-', '-', '+', '-'], 'постоянное место работы': ['+', '+', '-', '+', '-', '+', '+', '+', '+', '-', '+', '+'], 'поручитель': ['-', '-', '+', '+', '+', '-', '+', '-', '-', '+', '+', '+'], 'отсутствие задолженностей': ['+', '+', '+', '-', '+', '+', '-', '+', '+', '-', '+', '+'], 'доход выше 50k': ['+', '+', '-', '-', '-', '-', '+', '-', '+', '+', '+', '+'], 'постоянное место жительства': ['-', '-', '+', '-', '+', '-', '-', '+', '+', '+', '+', '+'], 'небольшая сумма займа': ['+', '+', '+', '-', '-', '-', '+', '-', '+', '-', '+', '+']})
example4 = example4.style.set_properties(**{'font-size': '16pt'})
example4

,высшее образование,постоянное место работы,поручитель,отсутствие задолженностей,доход выше 50k,постоянное место жительства,небольшая сумма займа
0,-,+,-,+,+,-,+
1,+,+,-,+,+,-,+
2,-,-,+,+,-,+,+
3,-,+,+,-,-,-,-
4,+,-,+,+,-,+,-
5,-,+,-,+,-,-,-
6,-,+,+,-,+,-,+
7,+,+,-,+,-,+,-
8,-,+,-,+,+,+,+
9,-,-,+,-,+,+,-


In [5]:
example5 = pd.DataFrame({'высшее образование': ['-', '-', '-', '-', '+', '+', '-', '-', '-', '-', '-', '+'], 'постоянное место работы': ['+', '-', '-', '-', '-', '-', '-', '-', '-', '-', '+', '-'], 'поручитель': ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'], 'отсутствие задолженностей': ['-', '-', '-', '-', '+', '+', '+', '+', '+', '-', '-', '-'], 'доход выше 50k': ['-', '+', '+', '-', '-', '-', '-', '-', '-', '-', '-', '-'], 'постоянное место жительства': ['-', '-', '-', '+', '-', '-', '+', '-', '-', '+', '-', '-'], 'небольшая сумма займа': ['-', '-', '+', '-', '+', '-', '-', '-', '+', '+', '+', '-']})
example5 = example5.style.set_properties(**{'font-size': '16pt'})
example5

,высшее образование,постоянное место работы,поручитель,отсутствие задолженностей,доход выше 50k,постоянное место жительства,небольшая сумма займа
0,-,+,-,-,-,-,-
1,-,-,-,-,+,-,-
2,-,-,-,-,+,-,+
3,-,-,-,-,-,+,-
4,+,-,-,+,-,-,+
5,+,-,-,+,-,-,-
6,-,-,-,+,-,+,-
7,-,-,-,+,-,-,-
8,-,-,-,+,-,-,+
9,-,-,-,-,-,+,+


#### Как и в прошлом примере, построив решетки и сформулировав гипотезы, получаем 6 положительных гипотез:
##### высшее образование И отсутствие задолженностей И постоянное место жительства
##### отсутствие задолженностей И постоянное место жительство И небольшая сумма займа
##### постоянное место работы И доход выше 50k И небольшая сумма займа
##### постоянное место работы И отсутствие задолженностей
##### поручитель
##### доход выше 50k И постоянное место жительства
#### Теперь рассмотрим новых клиентов, и будем либо одобрять займ, либо отказывать

In [7]:
example6 = pd.DataFrame({'высшее образование': ['-', '+', '+', '-', '-', '-', '-', '-'], 'постоянное место работы': ['+', '+', '-', '+', '+', '+', '+', '+'], 'поручитель': ['-', '-', '-', '-', '-', '-', '-', '-'], 'отсутствие задолженностей': ['-', '+', '+', '+', '-', '+', '+', '-'], 'доход выше 50k': ['+', '-', '-', '+', '-', '-', '-', '-'], 'постоянное место жительства': ['-', '-', '-', '-', '-', '-', '+', '+'], 'небольшая сумма займа': ['-', '-', '-', '-', '+', '+', '-', '+'], 'ЗАЙМ': ['отклонено', 'одобрено', 'отклонено', 'одобрено', 'отклонено', 'одобрено', 'одобрено', 'отклонено']})
example6 = example6.style.set_properties(**{'font-size': '16pt'})
example6

,высшее образование,постоянное место работы,поручитель,отсутствие задолженностей,доход выше 50k,постоянное место жительства,небольшая сумма займа,ЗАЙМ
0,-,+,-,-,+,-,-,отклонено
1,+,+,-,+,-,-,-,одобрено
2,+,-,-,+,-,-,-,отклонено
3,-,+,-,+,+,-,-,одобрено
4,-,+,-,-,-,-,+,отклонено
5,-,+,-,+,-,-,+,одобрено
6,-,+,-,+,-,+,-,одобрено
7,-,+,-,-,-,+,+,отклонено
